In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import load_img, img_to_array
from keras.applications import VGG19
from sklearn.model_selection import train_test_split

import numpy as np 

import os

In [2]:
def load_data(folder_name):
    X = []
    y = []
    class_names = os.listdir(folder_name)
    print(class_names)
    for i, class_name in enumerate(class_names):
        class_path = os.path.join(folder_name, class_name)
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            image = load_img(image_path, target_size = (224,224))
            image = img_to_array(image)
            X.append(image)
            y.append(i)
    X = np.array(X)
    y = np.array(y)
    return X, y

train_folder = "/kaggle/input/flowers-dataset/train"
X, y = load_data(train_folder)

['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']


In [3]:
X = X.astype('float32')/255
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)

In [6]:
#VGG19 architecture

model = Sequential([
    layers.Input(shape = (224, 224, 3)),
    
    layers.Conv2D(64, (3,3), padding = "same", activation = "relu"),
    layers.Conv2D(64, (3,3), padding = "same", activation = "relu"),
    layers.MaxPooling2D((2,2), strides = (2,2)),
                  
    layers.Conv2D(128, (3,3), padding = "same", activation = "relu"),
    layers.Conv2D(128, (3,3), padding = "same", activation = "relu"),
    layers.MaxPooling2D((2,2), strides = (2,2)),
                  
    layers.Conv2D(256, (3,3), padding = "same", activation = "relu"),
    layers.Conv2D(256, (3,3), padding = "same", activation = "relu"),
    layers.Conv2D(256, (3,3), padding = "same", activation = "relu"),
    layers.Conv2D(256, (3,3), padding = "same", activation = "relu"),
    layers.MaxPooling2D((2,2), strides = (2,2)),
                  
    layers.Conv2D(512, (3,3), padding = "same", activation = "relu"),
    layers.Conv2D(512, (3,3), padding = "same", activation = "relu"),
    layers.Conv2D(512, (3,3), padding = "same", activation = "relu"),
    layers.Conv2D(512, (3,3), padding = "same", activation = "relu"),
    layers.MaxPooling2D((2,2), strides = (2,2)),
                  
    layers.Conv2D(512, (3,3), padding = "same", activation = "relu"),
    layers.Conv2D(512, (3,3), padding = "same", activation = "relu"),
    layers.Conv2D(512, (3,3), padding = "same", activation = "relu"),
    layers.Conv2D(512, (3,3), padding = "same", activation = "relu"),
    layers.MaxPooling2D((2,2), strides = (2,2)),
                  
    Flatten(),
    Dense(4096, activation="relu"),
    Dense(4096, activation="relu"),
    Dense(1000, activation="relu"),
    
    Dense(5, activation="softmax")
])

In [8]:
model.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, validation_data = (X_val, y_val), batch_size = 32, epochs = 10)

In [12]:
#Using pre trained weights (Transfer Learning)

VGG19_weights = "/kaggle/input/vgg19-weights/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
VGG19_base = VGG19(weights=VGG19_weights, include_top=False, input_shape=(224, 224, 3))

model = Sequential()
model.add(VGG19_base)
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(256, activation = "relu"))
model.add(Dense(5, activation = "softmax"))

VGG19_base.trainable = False

In [13]:
model.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])

In [14]:
history = model.fit(X_train, y_train, validation_data = (X_val, y_val), batch_size = 32, epochs = 10)

Epoch 1/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 231s 3s/step - accuracy: 0.4432 - loss: 2.2574 - val_accuracy: 0.7364 - val_loss: 0.6796
Epoch 2/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 220s 3s/step - accuracy: 0.8227 - loss: 0.5155 - val_accuracy: 0.7636 - val_loss: 0.6794
Epoch 3/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 209s 3s/step - accuracy: 0.8993 - loss: 0.3190 - val_accuracy: 0.7982 - val_loss: 0.6111
Epoch 4/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 209s 3s/step - accuracy: 0.9351 - loss: 0.2257 - val_accuracy: 0.7709 - val_loss: 0.6363
Epoch 5/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 209s 3s/step - accuracy: 0.9416 - loss: 0.1710 - val_accuracy: 0.8127 - val_loss: 0.5933
Epoch 6/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 208s 3s/step - accuracy: 0.9621 - loss: 0.1244 - val_accuracy: 0.7945 - val_loss: 0.6434
Epoch 7/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 226s 3s/step - accuracy: 0.9708 - loss: 0.0992 - val_accuracy: 0.7745 - val_loss: 0.6892
Epoch 8/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 253s 4s/step - accuracy: 0.9782 - loss: 0.0831 - val_accuracy: 0.7909 - v